In [1]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [2]:
#import csv to df
file = Path('crypto_data.csv')
df = pd.read_csv(file)
df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [3]:
#Remove all rows that have at least one null value.
df.isnull().sum()

Unnamed: 0           0
CoinName             0
Algorithm            0
IsTrading            0
ProofType            0
TotalCoinsMined    508
TotalCoinSupply      0
dtype: int64

In [4]:
# Delete rows with null values
df = df.dropna()
df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...,...
1242,GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000


In [5]:
#Discard all cryptocurrencies that are not being traded. 
#filter for currencies that are currently being traded. 
trading_df = df[(df["IsTrading"] == True) ]
trading_df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,True,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [6]:
#Once you have done this, drop the IsTrading column from the dataframe.
clean_df = trading_df.drop(["IsTrading"], axis=1)
clean_df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [7]:
#Filter for cryptocurrencies that have been mined. That is, the total coins mined should be greater than zero.
mined_df = clean_df[(clean_df["TotalCoinsMined"] >= 1) ]
mined_df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [8]:
#In order for your dataset to be comprehensible to a machine learning algorithm, its data should be numeric. 
#Since the coin names do not contribute to the analysis of the data, 
#delete the CoinName from the original dataframe.
new_df = mined_df.drop(["CoinName", "Unnamed: 0"], axis='columns')
new_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2000000000
1242,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,CryptoNight,PoW,9.802226e+08,1400222610
1246,Equihash,PoW,7.296538e+06,21000000


In [9]:
#check data types
new_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [10]:
#change TotalCoinSupply to float
new_df['TotalCoinSupply'] = new_df['TotalCoinSupply'].astype(float)
new_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [11]:
#convert the remaining features with text values, Algorithm and ProofType, into numerical data. 
#use Pandas to create dummy variables
dummies_df = pd.get_dummies(new_df, columns = ['Algorithm', 'ProofType'])
dummies_df

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,2.000000e+09,2.000000e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1242,1.493105e+07,2.500000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1245,9.802226e+08,1.400223e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1246,7.296538e+06,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
column_list = dummies_df.columns.values.tolist()
column_list

['TotalCoinsMined',
 'TotalCoinSupply',
 'Algorithm_1GB AES Pattern Search',
 'Algorithm_536',
 'Algorithm_Argon2d',
 'Algorithm_BLAKE256',
 'Algorithm_Blake',
 'Algorithm_Blake2S',
 'Algorithm_Blake2b',
 'Algorithm_C11',
 'Algorithm_Cloverhash',
 'Algorithm_Counterparty',
 'Algorithm_CryptoNight',
 'Algorithm_CryptoNight Heavy',
 'Algorithm_CryptoNight-V7',
 'Algorithm_Cryptonight-GPU',
 'Algorithm_DPoS',
 'Algorithm_Dagger',
 'Algorithm_Dagger-Hashimoto',
 'Algorithm_ECC 256K1',
 'Algorithm_Equihash',
 'Algorithm_Equihash+Scrypt',
 'Algorithm_Ethash',
 'Algorithm_Exosis',
 'Algorithm_Green Protocol',
 'Algorithm_Groestl',
 'Algorithm_HMQ1725',
 'Algorithm_HybridScryptHash256',
 'Algorithm_IMesh',
 'Algorithm_Jump Consistent Hash',
 'Algorithm_Keccak',
 'Algorithm_Leased POS',
 'Algorithm_Lyra2RE',
 'Algorithm_Lyra2REv2',
 'Algorithm_Lyra2Z',
 'Algorithm_M7 POW',
 'Algorithm_Multiple',
 'Algorithm_NIST5',
 'Algorithm_NeoScrypt',
 'Algorithm_Ouroboros',
 'Algorithm_PHI1612',
 'Algorith

In [ ]:
# Examine the number of rows and columns of your dataset now. How did they change?
# went from 577 x 4 to 577 x 101 due to all the different values within 'Algorithm', 'ProofType'

In [ ]:

scaler = StandardScaler()
dummies_df[['TotalCoinSupply', 'TotalCoinsMined']] = scaler.fit_transform(dummies_df[['TotalCoinSupply', 'TotalCoinsMined']])
X = dummies_df


In [ ]:
X.shape

In [ ]:
X[:5]

In [ ]:
#Perform dimensionality reduction with PCA. 
#Rather than specify the number of principal components when you instantiate the PCA model, 
#it is possible to state the desired explained variance.

#For this project, preserve 90% of the explained variance in dimensionality reduction. 
#How did the number of the features change?
pca = PCA(n_components=.90)
crypto_pca = pca.fit_transform(X)



In [ ]:
pca.explained_variance_ratio_

In [ ]:
len(crypto_pca[0])

In [ ]:
#Next, further reduce the dataset dimensions with t-SNE and visually inspect the results. 
#In order to accomplish this task, run t-SNE on the principal components: the output of the PCA transformation. 
#Then create a scatter plot of the t-SNE output. Observe whether there are distinct clusters or not.

tsne = TSNE(learning_rate=200)
tsne_features = tsne.fit_transform(crypto_pca)

In [ ]:
 # Check that dimensions have been reduced
tsne_features.shape

In [ ]:
# Plot the results
plt.scatter(tsne_features[:,0], tsne_features[:,1])
plt.show()

In [ ]:
#Create an elbow plot to identify the best number of clusters. 
#Use a for-loop to determine the inertia for each k between 1 through 10. 
#Determine, if possible, where the elbow of the plot is, and at which value of k it appears.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(X)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.head()

In [ ]:
# Plot the elbow curve to find the best candidate(s) for k
plt.plot(df_elbow['k'], df_elbow['inertia'])
plt.xticks(range(1,11))
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow curve for Crypto')
plt.show()